# Building your LSTM RNN - Step by Step

Welcome to Course 5's first assignment! In this assignment, you will implement key components of a LSTM RNN in numpy.

Recurrent Neural Networks (RNN) are very effective for Natural Language Processing and other sequence tasks because they have "memory". They can read inputs $x^{\langle t \rangle}$ (such as words) one at a time, and remember some information/context through the hidden layer activations that get passed from one time-step to the next. This allows a unidirectional RNN to take information from the past to process later inputs. A bidirectional RNN can take context from both the past and the future. 

**Notation**:
- Superscript $[l]$ denotes an object associated with the $l^{th}$ layer. 

- Superscript $(i)$ denotes an object associated with the $i^{th}$ example. 

- Superscript $\langle t \rangle$ denotes an object at the $t^{th}$ time-step. 
    
- **Sub**script $i$ denotes the $i^{th}$ entry of a vector.

Example:  
- $a^{(2)[3]<4>}_5$ denotes the activation of the 2nd training example (2), 3rd layer [3], 4th time step <4>, and 5th entry in the vector.

#### Pre-requisites
* We assume that you are already familiar with `numpy`.  
* To refresh your knowledge of numpy, you can review course 1 of this specialization "Neural Networks and Deep Learning".  
    * Specifically, review the week 2 assignment ["Python Basics with numpy (optional)"](https://www.coursera.org/learn/neural-networks-deep-learning/item/Zh0CU).
    
    
Let's first import all the packages that you will need during this assignment.

In [1]:
import numpy as np
from rnn_utils import *

## 1. Long Short-Term Memory (LSTM) network

The following figure shows the operations of an LSTM-cell.

<img src="images/LSTM_figure4_v3a.png" style="width:500;height:400px;">
<caption><center><b>Figure 4</b>: LSTM-cell. This tracks and updates a "cell state" or memory variable $c^{\langle t \rangle}$ at every time-step, <br />which can be different from $a^{\langle t \rangle}$. </center></caption>

Similar to the RNN example above, you will start by implementing the LSTM cell for a single time-step. Then you can iteratively call it from inside a "for-loop" to have it process an input with $T_x$ time-steps. 

### Overview of gates and states

#### 1. Forget gate $\mathbf{\Gamma}_{f}$

- Let's assume we are reading words in a piece of text, and plan to use an LSTM to keep track of grammatical structures, such as whether the subject is singular ("puppy") or plural ("puppies"). 
- If the subject changes its state (from a singular word to a plural word), the memory of the previous state becomes outdated, so we "forget" that outdated state.
- The "forget gate" is a tensor containing values that are between 0 and 1.
  - If a unit in the forget gate has a value close to 0, the LSTM will "forget" the stored state in the corresponding unit of the previous cell state.
  - If a unit in the forget gate has a value close to 1, the LSTM will mostly remember the corresponding value in the stored state.


- **Equation**:

$$\mathbf{\Gamma}_f^{\langle t \rangle} = \sigma(\mathbf{W}_f[\mathbf{a}^{\langle t-1 \rangle},\  \mathbf{x}^{\langle t \rangle}] + \mathbf{b}_f)\tag{1} $$

- Explanation of the equation:

  - $\mathbf{W_{f}}$ contains weights that govern the forget gate's behavior. 
  - The previous time step's hidden state $[a^{\langle t-1 \rangle}$ and current time step's input $x^{\langle t \rangle}]$ are concatenated together and multiplied by $\mathbf{W_{f}}$. 
  - A sigmoid function is used to make each of the gate tensor's values $\mathbf{\Gamma}_f^{\langle t \rangle}$ range from 0 to 1.
  - The forget gate  $\mathbf{\Gamma}_f^{\langle t \rangle}$ has the same dimensions as the previous cell state $c^{\langle t-1 \rangle}$. 
  - This means that the two can be multiplied together, element-wise.
  - Multiplying the tensors $\mathbf{\Gamma}_f^{\langle t \rangle} * \mathbf{c}^{\langle t-1 \rangle}$ is like applying a mask over the previous cell state.
  - If a single value in $\mathbf{\Gamma}_f^{\langle t \rangle}$ is 0 or close to 0, then the product is close to 0.
    - This keeps the information stored in the corresponding unit in $\mathbf{c}^{\langle t-1 \rangle}$ from being remembered for the next time step.
  - Similarly, if one value is close to 1, the product is close to the original value in the previous cell state.
    - The LSTM will keep the information from the corresponding unit of $\mathbf{c}^{\langle t-1 \rangle}$, to be used in the next time step.


- Variable names in the code
  The variable names in the code are similar to the equations, with slight differences.  
  - `Wf` stands for the *forget gate weight*: $\mathbf{W}_{f}$
  - `Wb` stands for the *forget gate bias*: $\mathbf{W}_{b}$
  - `ft` stands for the *forget gate*: $\Gamma_f^{\langle t \rangle}$

<hr />

#### 2. Candidate value $\tilde{\mathbf{c}}^{\langle t \rangle}$
- The candidate value is a tensor containing information from the current time step that **may** be stored in the current cell state $\mathbf{c}^{\langle t \rangle}$.
- Which parts of the candidate value get passed on depends on the update gate.
- The candidate value is a tensor containing values that range from -1 to 1.
- The tilde "~" is used to differentiate the candidate $\tilde{\mathbf{c}}^{\langle t \rangle}$ from the cell state $\mathbf{c}^{\langle t \rangle}$.


- **Equation**
$$\mathbf{\tilde{c}}^{\langle t \rangle} = \tanh\left( \mathbf{W}_{c} [\mathbf{a}^{\langle t - 1 \rangle},\  \mathbf{x}^{\langle t \rangle}] + \mathbf{b}_{c} \right) \tag{3}$$

- Explanation of the equation
  - The 'tanh' function produces values between -1 and +1.


- Variable names in the code
  - `cct` stands for candidate value $\mathbf{\tilde{c}}^{\langle t \rangle}$

<hr />

#### 3. Update gate $\mathbf{\Gamma}_{i}$

* We use the update gate to decide what aspects of the candidate $\tilde{\mathbf{c}}^{\langle t \rangle}$ to add to the cell state $c^{\langle t \rangle}$.
* The update gate decides what parts of a "candidate" tensor $\tilde{\mathbf{c}}^{\langle t \rangle}$ are passed onto the cell state $\mathbf{c}^{\langle t \rangle}$.
* The update gate is a tensor containing values between 0 and 1.
    * When a unit in the update gate is close to 1, it allows the value of the candidate $\tilde{\mathbf{c}}^{\langle t \rangle}$ to be passed onto the hidden state $\mathbf{c}^{\langle t \rangle}$
    * When a unit in the update gate is close to 0, it prevents the corresponding value in the candidate from being passed onto the hidden state.
* Notice that we use the subscript "i" and not "u", to follow the convention used in the literature.


- **Equation**:

$$\mathbf{\Gamma}_i^{\langle t \rangle} = \sigma(\mathbf{W}_i[a^{\langle t-1 \rangle}, \mathbf{x}^{\langle t \rangle}] + \mathbf{b}_i)\tag{2} $$ 

- Explanation of the equation

  - Similar to the forget gate, here $\mathbf{\Gamma}_i^{\langle t \rangle}$, the sigmoid produces values between 0 and 1.
  - The update gate is multiplied element-wise with the candidate, and this product ($\mathbf{\Gamma}_{i}^{\langle t \rangle} * \tilde{c}^{\langle t \rangle}$) is used in determining the cell state $\mathbf{c}^{\langle t \rangle}$.

- Variable names in code (Please note that they're different than the equations)
  In the code, we'll use the variable names found in the academic literature.  These variables don't use "u" to denote "update".
  - `Wi` is the update gate weight $\mathbf{W}_i$ (not "Wu") 
  - `bi` is the update gate bias $\mathbf{b}_i$ (not "bu")
  - `it` is the forget gate $\mathbf{\Gamma}_i^{\langle t \rangle}$ (not "ut")

<hr />

#### 4. Cell state $\mathbf{c}^{\langle t \rangle}$

* The cell state is the "memory" that gets passed onto future time steps.
* The new cell state $\mathbf{c}^{\langle t \rangle}$ is a combination of the previous cell state and the candidate value.


- **Equation**:

$$ \mathbf{c}^{\langle t \rangle} = \mathbf{\Gamma}_f^{\langle t \rangle}* \mathbf{c}^{\langle t-1 \rangle} + \mathbf{\Gamma}_{i}^{\langle t \rangle} *\mathbf{\tilde{c}}^{\langle t \rangle} \tag{4} $$

- Explanation of equation
  - The previous cell state $\mathbf{c}^{\langle t-1 \rangle}$ is adjusted (weighted) by the forget gate $\mathbf{\Gamma}_{f}^{\langle t \rangle}$
  - and the candidate value $\tilde{\mathbf{c}}^{\langle t \rangle}$, adjusted (weighted) by the update gate $\mathbf{\Gamma}_{i}^{\langle t \rangle}$

- Variable names and shapes in the code
  - `c`: cell state, including all time steps, $\mathbf{c}$ shape $(n_{a},\ m,\ T)$
  - `c_next`: new (next) cell state, $\mathbf{c}^{\langle t \rangle}$ shape $(n_{a},\ m)$
  - `c_prev`: previous cell state, $\mathbf{c}^{\langle t-1 \rangle}$, shape $(n_{a},\ m)$
  
<hr />

#### 5. Output gate $\mathbf{\Gamma}_{o}$

- The output gate decides what gets sent as the prediction (output) of the time step.
- The output gate is like the other gates. It contains values that range from 0 to 1.


- **Equation**:

$$ \mathbf{\Gamma}_o^{\langle t \rangle}=  \sigma(\mathbf{W}_o[\mathbf{a}^{\langle t-1 \rangle}, \mathbf{x}^{\langle t \rangle}] + \mathbf{b}_{o})\tag{5}$$ 

- Explanation of the equation
  - The output gate is determined by the previous hidden state $\mathbf{a}^{\langle t-1 \rangle}$ and the current input $\mathbf{x}^{\langle t \rangle}$
  - The sigmoid makes the gate ranges from 0 to 1.


- Variable names in the code
  - `Wo`: output gate weight, $\mathbf{W_o}$
  - `bo`: output gate bias, $\mathbf{b_o}$
  - `ot`: output gate, $\mathbf{\Gamma}_{o}^{\langle t \rangle}$
  
<hr />

#### 6. Hidden state $\mathbf{a}^{\langle t \rangle}$

* The hidden state gets passed to the LSTM cell's next time step.
* It is used to determine the three gates ($\mathbf{\Gamma}_{f}, \mathbf{\Gamma}_{u}, \mathbf{\Gamma}_{o}$) of the next time step.
* The hidden state is also used for the prediction $y^{\langle t \rangle}$.


- **Equation**:

$$ \mathbf{a}^{\langle t \rangle} = \mathbf{\Gamma}_o^{\langle t \rangle} * \tanh(\mathbf{c}^{\langle t \rangle})\tag{6} $$

- Explanation of equation
  - The hidden state $\mathbf{a}^{\langle t \rangle}$ is determined by the cell state $\mathbf{c}^{\langle t \rangle}$ in combination with the output gate $\mathbf{\Gamma}_{o}$.
  - The cell state state is passed through the "tanh" function to rescale values between -1 and +1.
  - The output gate acts like a "mask" that either preserves the values of $\tanh(\mathbf{c}^{\langle t \rangle})$ or keeps those values from being included in the hidden state $\mathbf{a}^{\langle t \rangle}$

- Variable names  and shapes in the code
  - `a`: hidden state, including time steps.  $\mathbf{a}$ has shape $(n_{a}, m, T_{x})$
  - `a_prev`: hidden state from previous time step. $\mathbf{a}^{\langle t-1 \rangle}$ has shape $(n_{a}, m)$
  - `a_next`: hidden state for next time step.  $\mathbf{a}^{\langle t \rangle}$ has shape $(n_{a}, m)$ 
  
<hr />

#### 7. Prediction $\mathbf{y}^{\langle t \rangle}_{pred}$
- The prediction in this use case is a classification, so we'll use a softmax.


- **Equation**:
$$\mathbf{y}^{\langle t \rangle}_{pred} = \textrm{softmax}(\mathbf{W}_{y} \mathbf{a}^{\langle t \rangle} + \mathbf{b}_{y})$$

- Variable names and shapes in the code
  - `y_pred`: prediction, including all time steps. $\mathbf{y}_{pred}$ has shape $(n_{y}, m, T_{x})$.  Note that $(T_{y} = T_{x})$ for this example.
  - `yt_pred`: prediction for the current time step $t$. $\mathbf{y}^{\langle t \rangle}_{pred}$ has shape $(n_{y}, m)$

<hr />

### 1.1. LSTM cell

**Exercise**: Implement the LSTM cell described in the Figure (4).

**Instructions**:
1. Concatenate the hidden state $a^{\langle t-1 \rangle}$ and input $x^{\langle t \rangle}$ into a single matrix:  

$$concat = \begin{bmatrix} a^{\langle t-1 \rangle} \\ x^{\langle t \rangle} \end{bmatrix}$$  

2. Compute all the formulas 1 through 6 for the gates, hidden state, and cell state.
3. Compute the prediction $y^{\langle t \rangle}$.


#### Additional Hints
* You can use [numpy.concatenate](https://docs.scipy.org/doc/numpy/reference/generated/numpy.concatenate.html).  Check which value to use for the `axis` parameter.
* The functions `sigmoid()` and `softmax` are imported from `rnn_utils.py`.
* [np.tanh](https://docs.scipy.org/doc/numpy/reference/generated/numpy.tanh.html)
* Use [np.dot](https://docs.scipy.org/doc/numpy/reference/generated/numpy.dot.html) for matrix multiplication.
* Notice that the variable names `Wi`, `bi` refer to the weights and biases of the **update** gate.  There are no variables named "Wu" or "bu" in this function.

In [2]:
# GRADED FUNCTION: lstm_cell_forward

def lstm_cell_forward(xt, a_prev, c_prev, params):
    """
    Implement a single forward step of the LSTM-cell as described in Figure (4)

    Arguments:
    xt -- your input data at timestep "t", numpy array of shape (n_x, m).
    a_prev -- Hidden state at timestep "t-1", numpy array of shape (n_a, m)
    c_prev -- Memory state at timestep "t-1", numpy array of shape (n_a, m)
    params -- python dictionary containing:
                Wf -- Weight matrix of the forget gate, numpy array of shape (n_a, n_a + n_x)
                bf -- Bias of the forget gate, numpy array of shape (n_a, 1)
                Wi -- Weight matrix of the update gate, numpy array of shape (n_a, n_a + n_x)
                bi -- Bias of the update gate, numpy array of shape (n_a, 1)
                Wc -- Weight matrix of the first "tanh", numpy array of shape (n_a, n_a + n_x)
                bc --  Bias of the first "tanh", numpy array of shape (n_a, 1)
                Wo -- Weight matrix of the output gate, numpy array of shape (n_a, n_a + n_x)
                bo --  Bias of the output gate, numpy array of shape (n_a, 1)
                Wy -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_y, n_a)
                by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1)
                        
    Returns:
    a_next -- next hidden state, of shape (n_a, m)
    c_next -- next memory state, of shape (n_a, m)
    yt_pred -- prediction at timestep "t", numpy array of shape (n_y, m)
    cache -- tuple of values needed for the backward pass, contains (a_next, c_next, a_prev, c_prev, xt, params)
    
    Note: ft/it/ot stand for the forget/update/output gates, cct stands for the candidate value (c tilde),
          c stands for the cell state (memory)
    """

    # Retrieve parameters from "parameters"
    Wf = params["Wf"] # forget gate weight
    bf = params["bf"]
    Wi = params["Wi"] # update gate weight (notice the variable name)
    bi = params["bi"] # (notice the variable name)
    Wc = params["Wc"] # candidate value weight
    bc = params["bc"]
    Wo = params["Wo"] # output gate weight
    bo = params["bo"]
    Wy = params["Wy"] # prediction weight
    by = params["by"]
    
    # Retrieve dimensions from shapes of xt and Wy
    (n_x, m), (n_y, n_a) = xt.shape, Wy.shape

    ### CODE HERE:
    concat = np.concatenate((a_prev, xt), axis=0) # Concatenate a_prev and xt 

    # Calc. values for ft (forget gate), it (update gate), cct (candidate value), 
    # c_next (cell state), ot (output gate), a_next (hidden state)
    ft = sigmoid(Wf.dot(concat) + bf)             # forget gate
    it = sigmoid(Wi.dot(concat) + bi)             # update gate
    cct = np.tanh(Wc.dot(concat) + bc)            # candidate value
    c_next = ft * c_prev + it * cct               # cell state
    ot = sigmoid(Wo.dot(concat) + bo)             # output gate
    a_next = ot * np.tanh(c_next)                 # hidden state
    yt_pred = softmax(Wy.dot(a_next) + by)        # Calc. prediction of the LSTM cell
    
    # store values needed for backward propagation in cache
    cache = (a_next, c_next, a_prev, c_prev, ft, it, cct, ot, xt, params)

    return a_next, c_next, yt_pred, cache

In [11]:
np.random.seed(1)
xt_tmp = np.random.randn(3, 10)
a_prev_tmp = np.random.randn(5, 10)
c_prev_tmp = np.random.randn(5, 10)

params_tmp = {}
params_tmp['Wf'] = np.random.randn(5, 5+3)
params_tmp['bf'] = np.random.randn(5,1)
params_tmp['Wi'] = np.random.randn(5, 5+3)
params_tmp['bi'] = np.random.randn(5,1)
params_tmp['Wo'] = np.random.randn(5, 5+3)
params_tmp['bo'] = np.random.randn(5,1)
params_tmp['Wc'] = np.random.randn(5, 5+3)
params_tmp['bc'] = np.random.randn(5,1)
params_tmp['Wy'] = np.random.randn(2,5)
params_tmp['by'] = np.random.randn(2,1)

a_next_tmp, c_next_tmp, yt_tmp, cache_tmp = lstm_cell_forward(xt_tmp, a_prev_tmp, c_prev_tmp, params_tmp)
print("a_next[4] = \n", a_next_tmp[4])
print("a_next.shape = ", c_next_tmp.shape)
print()
print("c_next[2] = \n", c_next_tmp[2])
print("c_next.shape = ", c_next_tmp.shape)
print()
print("yt[1] =", yt_tmp[1])
print("yt.shape = ", yt_tmp.shape)
print()
print("cache[1][3] =\n", cache_tmp[1][3])
print("len(cache) = ", len(cache_tmp))

a_next[4] = 
 [-0.66408471  0.0036921   0.02088357  0.22834167 -0.85575339  0.00138482
  0.76566531  0.34631421 -0.00215674  0.43827275]
a_next.shape =  (5, 10)

c_next[2] = 
 [ 0.63267805  1.00570849  0.35504474  0.20690913 -1.64566718  0.11832942
  0.76449811 -0.0981561  -0.74348425 -0.26810932]
c_next.shape =  (5, 10)

yt[1] = [0.79913913 0.15986619 0.22412122 0.15606108 0.97057211 0.31146381
 0.00943007 0.12666353 0.39380172 0.07828381]
yt.shape =  (2, 10)

cache[1][3] =
 [-0.16263996  1.03729328  0.72938082 -0.54101719  0.02752074 -0.30821874
  0.07651101 -1.03752894  1.41219977 -0.37647422]
len(cache) =  10


**Expected Output**:

```Python
a_next[4] = 
 [-0.66408471  0.0036921   0.02088357  0.22834167 -0.85575339  0.00138482
  0.76566531  0.34631421 -0.00215674  0.43827275]
a_next.shape =  (5, 10)

c_next[2] = 
 [ 0.63267805  1.00570849  0.35504474  0.20690913 -1.64566718  0.11832942
  0.76449811 -0.0981561  -0.74348425 -0.26810932]
c_next.shape =  (5, 10)

yt[1] = [ 0.79913913  0.15986619  0.22412122  0.15606108  0.97057211  0.31146381
  0.00943007  0.12666353  0.39380172  0.07828381]
yt.shape =  (2, 10)

cache[1][3] =
 [-0.16263996  1.03729328  0.72938082 -0.54101719  0.02752074 -0.30821874
  0.07651101 -1.03752894  1.41219977 -0.37647422]
len(cache) =  10
```

### 1.2. Forward pass for LSTM

Now that you have implemented one step of an LSTM, you can now iterate this over this using a for-loop to process a sequence of $T_x$ inputs. 

<img src="images/LSTM_rnn.png" style="width:500;height:300px;">
<caption><center> **Figure 5**: LSTM over multiple time-steps. </center></caption>

**Exercise:** Implement `lstm_forward()` to run an LSTM over $T_x$ time-steps. 

**Instructions**
* Get the dimensions $n_x, n_a, n_y, m, T_x$ from the shape of the variables: `x` and `parameters`.
* Initialize the 3D tensors $a$, $c$ and $y$.
    - $a$: hidden state, shape $(n_{a}, m, T_{x})$
    - $c$: cell state, shape $(n_{a}, m, T_{x})$
    - $y$: prediction, shape $(n_{y}, m, T_{x})$ (Note that $T_{y} = T_{x}$ in this example).
    - **Note** Setting one variable equal to the other is a "copy by reference" (which means both reference points to same memory location).
* Initialize the 2D tensor $a^{\langle t \rangle}$ 
    - $a^{\langle t \rangle}$ stores the hidden state for time step $t$.  The variable name is `a_next`.
    - $a^{\langle 0 \rangle}$, the initial hidden state at time step 0, is passed in when calling the function. The variable name is `a0`.
    - $a^{\langle t \rangle}$ and $a^{\langle 0 \rangle}$ represent a single time step, so they both have the shape  $(n_{a}, m)$ 
    - Initialize $a^{\langle t \rangle}$ by setting it to the initial hidden state ($a^{\langle 0 \rangle}$) that is passed into the function.
* Initialize $c^{\langle t \rangle}$ with zeros. 
    - The variable name is `c_next`. 
    - $c^{\langle t \rangle}$ represents a single time step, so its shape is $(n_{a}, m)$
    - **Note**: create `c_next` as its own variable with its own location in memory.  Do not initialize it as a slice of the 3D tensor $c$.  In other words, **don't** do `c_next = c[:,:,0]`.
* For each time step, do the following:
    - From the 3D tensor $x$, get a 2D slice $x^{\langle t \rangle}$ at time step $t$.
    - Call the `lstm_cell_forward` function that you defined previously, to get the hidden state, cell state, prediction, and cache.
    - Store the hidden state, cell state and prediction (the 2D tensors) inside the 3D tensors.
    - Also append the cache to the list of caches.

In [4]:
# GRADED FUNCTION: lstm_forward

def lstm_forward(x, a0, params):
    """
    Implement the forward propagation of the RNN using an LSTM-cell described in Figure (4).

    Arguments:
    x -- Input data for every time-step, of shape (n_x, m, T_x).
    a0 -- Initial hidden state, of shape (n_a, m)
    params -- python dictionary containing:
                Wf -- Weight matrix of the forget gate, numpy array of shape (n_a, n_a + n_x)
                bf -- Bias of the forget gate, numpy array of shape (n_a, 1)
                Wi -- Weight matrix of the update gate, numpy array of shape (n_a, n_a + n_x)
                bi -- Bias of the update gate, numpy array of shape (n_a, 1)
                Wc -- Weight matrix of the first "tanh", numpy array of shape (n_a, n_a + n_x)
                bc -- Bias of the first "tanh", numpy array of shape (n_a, 1)
                Wo -- Weight matrix of the output gate, numpy array of shape (n_a, n_a + n_x)
                bo -- Bias of the output gate, numpy array of shape (n_a, 1)
                Wy -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_y, n_a)
                by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1)
                        
    Returns:
    a -- Hidden states for every time-step, numpy array of shape (n_a, m, T_x)
    y -- Predictions for every time-step, numpy array of shape (n_y, m, T_x)
    c -- The value of the cell state, numpy array of shape (n_a, m, T_x)
    caches -- tuple of values needed for the backward pass, contains (list of all the caches, x)
    """

    caches = [] # Initialize "caches" list
    
    ### CODE HERE:
    (n_x, m, T_x), (n_y, n_a) = x.shape, params['Wy'].shape # Retrieve dimensions from shapes of x and params['Wy']
    
    # initialize "a", "c" and "y" with zeros
    a, c = np.zeros((n_a, m, T_x)), np.zeros((n_a, m, T_x))
    y = np.zeros((n_y, m, T_x))
    
    # Initialize a_next and c_next
    a_next, c_next = a0, np.zeros((n_a, m))
    
    # loop over all time-steps
    for t in range(T_x):
        xt = x[:, :, t] # Get the 2D slice 'xt' from the 3D input 'x' at time step 't'
        
        # Update next hidden state, next memory state, compute the prediction, get the cache
        a_next, c_next, yt, cache = lstm_cell_forward(xt, a_next, c_next, params)
        
        a[:, :, t] = a_next  # Save the value of the new "next" hidden state
        c[:, :, t] = c_next  # Save the value of the next cell state
        y[:, :, t] = yt      # Save the value of the prediction
        caches.append(cache) # Append the cache into caches
    #
    caches = (caches, x) # store values needed for backward propagation in cache

    return a, y, c, caches

In [14]:
np.random.seed(1)
x_tmp = np.random.randn(3, 10, 7)
a0_tmp = np.random.randn(5, 10)

params_tmp = {}
params_tmp['Wf'] = np.random.randn(5, 5+3)
params_tmp['bf'] = np.random.randn(5,1)
params_tmp['Wi'] = np.random.randn(5, 5+3)
params_tmp['bi']= np.random.randn(5,1)
params_tmp['Wo'] = np.random.randn(5, 5+3)
params_tmp['bo'] = np.random.randn(5,1)
params_tmp['Wc'] = np.random.randn(5, 5+3)
params_tmp['bc'] = np.random.randn(5,1)
params_tmp['Wy'] = np.random.randn(2,5)
params_tmp['by'] = np.random.randn(2,1)

a_tmp, y_tmp, c_tmp, caches_tmp = lstm_forward(x_tmp, a0_tmp, params_tmp)
print("a[4][3][6] =", a_tmp[4][3][6])
print("a.shape =", a_tmp.shape)
print()
print("y[1][4][3] =", y_tmp[1][4][3])
print("y.shape =", y_tmp.shape)
print()
print("caches[1][1][1] =\n", caches_tmp[1][1][1])
print("c[1][2][1] =", c_tmp[1][2][1])
print("len(caches) =", len(caches_tmp))

a[4][3][6] = 0.17211776753291672
a.shape = (5, 10, 7)

y[1][4][3] = 0.9508734618501101
y.shape = (2, 10, 7)

caches[1][1][1] =
 [ 0.82797464  0.23009474  0.76201118 -0.22232814 -0.20075807  0.18656139
  0.41005165]
c[1][2][1] = -0.8555449167181981
len(caches) = 2


**Expected Output**:

```Python
a[4][3][6] =  0.172117767533
a.shape = (5, 10, 7)

y[1][4][3] = 0.95087346185
y.shape = (2, 10, 7)

caches[1][1][1] =
 [ 0.82797464  0.23009474  0.76201118 -0.22232814 -0.20075807  0.18656139
  0.41005165]
c[1][2][1] = -0.855544916718
len(caches) = 2
```

Congratulations! You have now implemented the forward passes for the basic RNN and the LSTM. When using a deep learning framework, implementing the forward pass is sufficient to build systems that achieve great performance. 

## 2. Backpropagation in recurrent neural networks 

In modern deep learning frameworks, you only have to implement the forward pass, and the framework takes care of the backward pass, so most deep learning engineers do not need to bother with the details of the backward pass. If however you are an expert in calculus and want to see the details of backprop in RNNs, you can work through this optional portion of the notebook. 

When in an earlier [course](https://www.coursera.org/learn/neural-networks-deep-learning/lecture/0VSHe/derivatives-with-a-computation-graph)  you implemented a simple (fully connected) neural network, you used backpropagation to compute the derivatives with respect to the cost to update the parameters. Similarly, in recurrent neural networks you can calculate the derivatives with respect to the cost in order to update the parameters. The backprop equations are quite complicated and we did not derive them in lecture. However, we will briefly present them below. 

*Note* that this notebook does NOT implement the backward path from the Loss 'J' backwards to 'a'. This would have included the dense layer and softmax which are a part of the forward path. This is assumed to be calculated elsewhere and the result passed to rnn_backward in 'da'. It is further assumed that loss has been adjusted for batch size (m) and division by the number of examples is not required here.

## 2.1. LSTM backward pass

### 2.1.1 One Step backward
The LSTM backward pass is slighltly more complicated than the forward one.

<img src="images/LSTM_cell_backward_rev3a_5.png" style="width:500;height:400px;"> <br>
<caption><center><b>Figure 8</b>: lstm_cell_backward. Note the output functions, while part of the lstm_cell_forward, are not included in lstm_cell_backward </center></caption>


The equations for the LSTM backward pass are provided below. (If you enjoy calculus exercises feel free to try deriving these from scratch yourself.)

### 2.1.2. Gate derivatives
Note the location of the gate derivatives ($\gamma$..) between the dense layer and the activation function (see graphic above).  
This is convenient for computing parameter derivatives in the next step. 

$d\gamma_o^{\langle t \rangle} = da_{next}*\tanh(c_{next}) * \Gamma_o^{\langle t \rangle}*\left(1-\Gamma_o^{\langle t \rangle}\right)\tag{7}$

$d\widetilde{c}^{\langle t \rangle} = \left(dc_{next}*\Gamma_u^{\langle t \rangle}+ \Gamma_o^{\langle t \rangle}* (1-\tanh^2(c_{next})) * \Gamma_u^{\langle t \rangle} * da_{next} \right) * \left(1-\left(\widetilde c^{\langle t \rangle}\right)^2\right) \tag{8}$

$d\gamma_u^{\langle t \rangle} = \left(dc_{next}*\widetilde{c}^{\langle t \rangle} + \Gamma_o^{\langle t \rangle}* (1-\tanh^2(c_{next})) * \widetilde{c}^{\langle t \rangle} * da_{next}\right)*\Gamma_u^{\langle t \rangle}*\left(1-\Gamma_u^{\langle t \rangle}\right)\tag{9}$

$d\gamma_f^{\langle t \rangle} = \left(dc_{next}* c_{prev} + \Gamma_o^{\langle t \rangle} * (1-\tanh^2(c_{next})) * c_{prev} * da_{next}\right)*\Gamma_f^{\langle t \rangle}*\left(1-\Gamma_f^{\langle t \rangle}\right)\tag{10}$

### 2.1.3. Parameter derivatives 

$ dW_f = d\gamma_f^{\langle t \rangle} \begin{bmatrix} a_{prev} \\ x_t\end{bmatrix}^T \tag{11} $
$ dW_u = d\gamma_u^{\langle t \rangle} \begin{bmatrix} a_{prev} \\ x_t\end{bmatrix}^T \tag{12} $
$ dW_c = d\widetilde c^{\langle t \rangle} \begin{bmatrix} a_{prev} \\ x_t\end{bmatrix}^T \tag{13} $
$ dW_o = d\gamma_o^{\langle t \rangle} \begin{bmatrix} a_{prev} \\ x_t\end{bmatrix}^T \tag{14}$

To calculate $db_f, db_u, db_c, db_o$ you just need to sum across the horizontal (axis= 1) axis on $d\gamma_f^{\langle t \rangle}, d\gamma_u^{\langle t \rangle}, dp\widetilde c^{\langle t \rangle}, d\gamma_o^{\langle t \rangle}$ respectively.  
Note that you should use the `keepdims = True` option.

$\displaystyle db_f = \sum_{batch} d\gamma_f^{\langle t \rangle} \tag{15}$
$\displaystyle db_u = \sum_{batch} d\gamma_u^{\langle t \rangle} \tag{16}$
$\displaystyle db_c = \sum_{batch} d\widetilde c^{\langle t \rangle} \tag{17}$
$\displaystyle db_o = \sum_{batch} d\gamma_o^{\langle t \rangle} \tag{18}$

Finally, you will compute the derivative with respect to the previous hidden state, previous memory state, and input.

$ da_{prev} = W_f^T d\gamma_f^{\langle t \rangle} + W_u^T   d\gamma_u^{\langle t \rangle}+ W_c^T dp\widetilde c^{\langle t \rangle} + W_o^T d\gamma_o^{\langle t \rangle} \tag{19}$

Here, to account for concatenation, the weights for equations 19 are the first n_a, (i.e. $W_f = W_f[:,:n_a]$ etc...)

$ dc_{prev} = dc_{next}*\Gamma_f^{\langle t \rangle} + \Gamma_o^{\langle t \rangle} * (1- \tanh^2(c_{next}))*\Gamma_f^{\langle t \rangle}*da_{next} \tag{20}$

$ dx^{\langle t \rangle} = W_f^T d\gamma_f^{\langle t \rangle} + W_u^T  d\gamma_u^{\langle t \rangle}+ W_c^T d\widetilde c^{\langle t \rangle} + W_o^T d\gamma_o^{\langle t \rangle}\tag{21} $

where the weights for equation 21 are from n_a to the end, (i.e. $W_f = W_f[:,n_a:]$ etc...)

**Exercise:** Implement `lstm_cell_backward` by implementing equations $7-21$ below. 
  
    
Note: In the code:

$d\gamma_o^{\langle t \rangle}$ is represented by  `dot_` (*avoid possible clash with `np.dot()`*),    
$d\widetilde{c}^{\langle t \rangle}$ is represented by  `dcct`,  
$d\gamma_u^{\langle t \rangle}$ is represented by  `dit`,  
$d\gamma_f^{\langle t \rangle}$ is represented by  `dft`


In [6]:
def lstm_cell_backward(da_next, dc_next, cache):
    """
    Implement the backward pass for the LSTM-cell (single time-step).

    Arguments:
    da_next -- Gradients of next hidden state, of shape (n_a, m)
    dc_next -- Gradients of next cell state, of shape (n_a, m)
    cache -- cache storing information from the forward pass

    Returns:
    gradients -- python dictionary containing:
                   dxt -- Gradient of input data at time-step t, of shape (n_x, m)
                   da_prev -- Gradient w.r.t. the previous hidden state, numpy array of shape (n_a, m)
                   dc_prev -- Gradient w.r.t. the previous memory state, of shape (n_a, m, T_x)
                   dWf -- Gradient w.r.t. weight matrix of the forget gate, np array of shape (n_a, n_a + n_x)
                   dWi -- Gradient w.r.t. weight matrix of the update gate, np array of shape (n_a, n_a + n_x)
                   dWc -- Gradient w.r.t. weight matrix of the memory gate, np array of shape (n_a, n_a + n_x)
                   dWo -- Gradient w.r.t. weight matrix of the output gate, np array of shape (n_a, n_a + n_x)
                   dbf -- Gradient w.r.t. biases of the forget gate, of shape (n_a, 1)
                   dbi -- Gradient w.r.t. biases of the update gate, of shape (n_a, 1)
                   dbc -- Gradient w.r.t. biases of the memory gate, of shape (n_a, 1)
                   dbo -- Gradient w.r.t. biases of the output gate, of shape (n_a, 1)
    """
    # Retrieve information from "cache"
    (a_next, c_next, a_prev, c_prev, ft, it, cct, ot, xt, params) = cache
    
    ### CODE HERE: 
    (n_x, m), (n_a, m ) = xt.shape, a_next.shape # Retrieve dimensions from xt's and a_next's shape
    
    assert da_next.shape == (n_a, m) # da_next.shape:  (5, 10)
    assert dc_next.shape == (n_a, m) 
    
    # Retrieve parameters from "params", convenience for eq 
    Wf = params["Wf"] # forget gate weight, shape: (n_a, n_a + n_x) == (5, 8)
    assert Wf.shape == (n_a, n_a + n_x)
    
    Wi = params["Wi"] # update gate weight (notice the variable name), shape: (n_a, n_a + n_x) == (5, 8)
    assert Wi.shape == (n_a, n_a + n_x)
    
    Wc = params["Wc"] # candidate value weight, shape: (n_a, n_a + n_x) == (5, 8)
    assert Wc.shape == (n_a, n_a + n_x)
    
    Wo = params["Wo"] # output gate weight, shape: (n_a, n_a + n_x) == (5, 8)
    assert Wo.shape == (n_a, n_a + n_x)
    
    # Compute gates related derivatives, according to equations (7) to (10)
    z = (1 - np.tanh(c_next)**2)
    dot_ = da_next * np.tanh(c_next) * ot * (1 - ot)                      # eq (7)
    assert dot_.shape == da_next.shape # == (5, 10)
    
    dcct = (dc_next * it + ot * z * it * da_next) * (1 - cct**2)          # eq (8)
    assert dcct.shape == da_next.shape # == (5, 10)
    
    dit  = (dc_next * cct + ot * z * cct * da_next) * it * (1 - it)       # eq (9)
    assert dit.shape == da_next.shape  # == (5, 10)
    
    dft  = (dc_next * c_prev + ot * z * c_prev * da_next) * ft * (1 - ft) # eq (10)
    assert dft.shape == da_next.shape  # == (5, 10)
    
    # Compute parameters related derivatives. Use equations (11)-(18) 
    aprev_xt = np.concatenate((a_prev, xt), axis=0).T # Concatenate a_prev and xt, then transpose
    # a_prev.shape = (n_a: 5, m: 10), xt.shape = (n_x: 3, m: 10) ==> aprev_xt(T).shape == (m: 10, n_a + n_x: 8)     
    dWf = dft.dot(aprev_xt)    # eq (11) (n_a, m) x (m, n_a + n_x) => (n_a, n_a + n_x)
    assert dWf.shape == (n_a, n_a + n_x) 
    
    dWi = dit.dot(aprev_xt)    # eq (12)
    assert dWi.shape == (n_a, n_a + n_x) 
    
    dWc = dcct.dot(aprev_xt)   # eq (13)
    assert dWc.shape == (n_a, n_a + n_x) 
    
    dWo = dot_.dot(aprev_xt)   # eq (14)
    assert dWo.shape == (n_a, n_a + n_x) 
    
    dbf = np.sum(dft, axis=1, keepdims=True)    # eq (15)
    assert dbf.shape == (n_a, 1)
    
    dbi = np.sum(dit, axis=1, keepdims=True)    # eq (16)
    assert dbi.shape == (n_a, 1)
    
    dbc = np.sum(dcct, axis=1, keepdims=True)   # eq (17)
    assert dbc.shape == (n_a, 1)
    
    dbo = np.sum(dot_, axis=1, keepdims=True)   # eq (18)
    assert dbo.shape == (n_a, 1)

    # Compute derivatives w.r.t prev. hidden state, prev. memory state and input. 
    # Use equations (19)-(21). 
    da_prev = Wf[:, :n_a].T.dot(dft) + Wi[:, :n_a].T.dot(dit) \
        + Wc[:, :n_a].T.dot(dcct) + Wo[:, :n_a].T.dot(dot_)  # eq (19)
    assert da_prev.shape == (n_a, m)
    
    dc_prev = dc_next * ft + ot * z * ft * da_next                             # eq (20)
    assert dc_prev.shape == (n_a, m)
    
    dxt     = Wf[:, n_a:].T.dot(dft) + Wi[:, n_a:].T.dot(dit) + \
        Wc[:, n_a:].T.dot(dcct) + Wo[:, n_a:].T.dot(dot_)                      # eq (21)
    assert dxt.shape == (n_x, m)
    
    # Save gradients in dictionary
    gradients = {"dxt": dxt, "da_prev": da_prev, "dc_prev": dc_prev, "dWf": dWf,"dbf": dbf, "dWi": dWi, 
                 "dbi": dbi, "dWc": dWc, "dbc": dbc, "dWo": dWo, "dbo": dbo}

    return gradients

In [7]:
np.random.seed(1)
xt_tmp = np.random.randn(3, 10)
a_prev_tmp = np.random.randn(5, 10)
c_prev_tmp = np.random.randn(5, 10)

params_tmp = {}
params_tmp['Wf'] = np.random.randn(5, 5 + 3)
params_tmp['bf'] = np.random.randn(5, 1)
params_tmp['Wi'] = np.random.randn(5, 5 + 3)
params_tmp['bi'] = np.random.randn(5, 1)
params_tmp['Wo'] = np.random.randn(5, 5 + 3)
params_tmp['bo'] = np.random.randn(5, 1)
params_tmp['Wc'] = np.random.randn(5, 5 + 3)
params_tmp['bc'] = np.random.randn(5, 1)
params_tmp['Wy'] = np.random.randn(2, 5)
params_tmp['by'] = np.random.randn(2, 1)

a_next_tmp, c_next_tmp, yt_tmp, cache_tmp = lstm_cell_forward(xt_tmp, a_prev_tmp, c_prev_tmp, params_tmp)

da_next_tmp = np.random.randn(5, 10)
dc_next_tmp = np.random.randn(5, 10)
gradients_tmp = lstm_cell_backward(da_next_tmp, dc_next_tmp, cache_tmp)

print("gradients[\"dxt\"][1][2] =", gradients_tmp["dxt"][1][2])
print("gradients[\"dxt\"].shape =", gradients_tmp["dxt"].shape)
print("gradients[\"da_prev\"][2][3] =", gradients_tmp["da_prev"][2][3])
print("gradients[\"da_prev\"].shape =", gradients_tmp["da_prev"].shape)
print("gradients[\"dc_prev\"][2][3] =", gradients_tmp["dc_prev"][2][3])
print("gradients[\"dc_prev\"].shape =", gradients_tmp["dc_prev"].shape)
print()
print("gradients[\"dWf\"][3][1] =", gradients_tmp["dWf"][3][1])
print("gradients[\"dWf\"].shape =", gradients_tmp["dWf"].shape)
print("gradients[\"dWi\"][1][2] =", gradients_tmp["dWi"][1][2])
print("gradients[\"dWi\"].shape =", gradients_tmp["dWi"].shape)
print("gradients[\"dWc\"][3][1] =", gradients_tmp["dWc"][3][1])
print("gradients[\"dWc\"].shape =", gradients_tmp["dWc"].shape)
print("gradients[\"dWo\"][1][2] =", gradients_tmp["dWo"][1][2])
print("gradients[\"dWo\"].shape =", gradients_tmp["dWo"].shape)
print()
print("gradients[\"dbf\"][4] =", gradients_tmp["dbf"][4])
print("gradients[\"dbf\"].shape =", gradients_tmp["dbf"].shape)
print("gradients[\"dbi\"][4] =", gradients_tmp["dbi"][4])
print("gradients[\"dbi\"].shape =", gradients_tmp["dbi"].shape)
print("gradients[\"dbc\"][4] =", gradients_tmp["dbc"][4])
print("gradients[\"dbc\"].shape =", gradients_tmp["dbc"].shape)
print("gradients[\"dbo\"][4] =", gradients_tmp["dbo"][4])
print("gradients[\"dbo\"].shape =", gradients_tmp["dbo"].shape)

gradients["dxt"][1][2] = 3.2305591151091875
gradients["dxt"].shape = (3, 10)
gradients["da_prev"][2][3] = -0.06396214197109236
gradients["da_prev"].shape = (5, 10)
gradients["dc_prev"][2][3] = 0.7975220387970015
gradients["dc_prev"].shape = (5, 10)

gradients["dWf"][3][1] = -0.1479548381644968
gradients["dWf"].shape = (5, 8)
gradients["dWi"][1][2] = 1.0574980552259903
gradients["dWi"].shape = (5, 8)
gradients["dWc"][3][1] = 2.3045621636876668
gradients["dWc"].shape = (5, 8)
gradients["dWo"][1][2] = 0.3313115952892109
gradients["dWo"].shape = (5, 8)

gradients["dbf"][4] = [0.18864637]
gradients["dbf"].shape = (5, 1)
gradients["dbi"][4] = [-0.40142491]
gradients["dbi"].shape = (5, 1)
gradients["dbc"][4] = [0.25587763]
gradients["dbc"].shape = (5, 1)
gradients["dbo"][4] = [0.13893342]
gradients["dbo"].shape = (5, 1)


**Expected Output**:

<table>
    <tr>
        <td>
            <b>gradients["dxt"][1][2]</b>=
        </td>
        <td>
           3.23055911511
        </td>
    </tr>
        <tr>
        <td>
            <b>gradients["dxt"].shape</b>=
        </td>
        <td>
           (3, 10)
        </td>
    </tr>
        <tr>
        <td>
            <b>gradients["da_prev"][2][3]</b>=
        </td>
        <td>
           -0.0639621419711
        </td>
    </tr>
        <tr>
        <td>
            <b>gradients["da_prev"].shape</b>=
        </td>
        <td>
           (5, 10)
        </td>
    </tr>
         <tr>
        <td>
            <b>gradients["dc_prev"][2][3]</b>=
        </td>
        <td>
           0.797522038797
        </td>
    </tr>
        <tr>
        <td>
            <b>gradients["dc_prev"].shape</b>=
        </td>
        <td>
           (5, 10)
        </td>
    </tr>
    <tr><td>&nbsp;</td></tr>
        <tr>
        <td>
            <b>gradients["dWf"][3][1]</b>= 
        </td>
        <td>
           -0.147954838164
        </td>
    </tr>
        <tr>
        <td>
            <b>gradients["dWf"].shape</b>=
        </td>
        <td>
           (5, 8)
        </td>
    </tr>
        <tr>
        <td>
            <b>gradients["dWi"][1][2]</b>= 
        </td>
        <td>
           1.05749805523
        </td>
    </tr>
        <tr>
        <td>
            <b>gradients["dWi"].shape</b>= 
        </td>
        <td>
           (5, 8)
        </td>
    </tr>
    <tr>
        <td>
            <b>gradients["dWc"][3][1]</b>= 
        </td>
        <td>
           2.30456216369
        </td>
    </tr>
        <tr>
        <td>
            <b>gradients["dWc"].shape</b>= 
        </td>
        <td>
           (5, 8)
        </td>
    </tr>
    <tr>
        <td>
            <b>gradients["dWo"][1][2]</b>= 
        </td>
        <td>
           0.331311595289
        </td>
    </tr>
        <tr>
        <td>
            <b>gradients["dWo"].shape</b>= 
        </td>
        <td>
           (5, 8)
        </td>
    </tr>
    <tr><td>&nbsp;</td></tr>
    <tr>
        <td>
            <b>gradients["dbf"][4]</b>= 
        </td>
        <td>
           [ 0.18864637]
        </td>
    </tr>
        <tr>
        <td>
            <b>gradients["dbf"].shape</b>= 
        </td>
        <td>
           (5, 1)
        </td>
    </tr>
    <tr>
        <td>
            <b>gradients["dbi"][4]</b>= 
        </td>
        <td>
           [-0.40142491]
        </td>
    </tr>
        <tr>
        <td>
            <b>gradients["dbi"].shape</b>= 
        </td>
        <td>
           (5, 1)
        </td>
    </tr>
        <tr>
        <td>
            <b>gradients["dbc"][4]</b>= 
        </td>
        <td>
           [ 0.25587763]
        </td>
    </tr>
        <tr>
        <td>
            <b>gradients["dbc"].shape</b>= 
        </td>
        <td>
           (5, 1)
        </td>
    </tr>
        <tr>
        <td>
            <b>gradients["dbo"][4]</b>= 
        </td>
        <td>
           [ 0.13893342]
        </td>
    </tr>
        <tr>
        <td>
            <b>gradients["dbo"].shape</b>= 
        </td>
        <td>
           (5, 1)
        </td>
    </tr>
</table>

### 2.2. Backward pass through the LSTM RNN

This part is very similar to the `rnn_backward` function you implemented (cf. notebook *Building_vanilla_RNN-Step_by_Step.ipynb*).

1. Create variables of the same dimension as your return variables. 
2. Iterate over all the time steps starting from the end and call the one step function implemented for LSTM at each iteration. 
3. Update the parameters by summing them individually. 
4. Return a dictionary with the new gradients. 

**Instructions**: Implement the `lstm_backward` function. 
- Create a for loop starting from $T_x$ and going backward. 
- For each step call `lstm_cell_backward` and update the old gradients by adding the new gradients to them. 

*Note* that `dxt` is not updated but is stored.

In [89]:
def lstm_backward(da, caches):
    
    """
    Implement the backward pass for the RNN with LSTM-cell (over a whole sequence).

    Arguments:
    da -- Gradients w.r.t the hidden states, numpy-array of shape (n_a, m, T_x)
    caches -- cache storing information from the forward pass (lstm_forward)

    Returns:
    gradients -- python dictionary containing:
                   dx -- Gradient of inputs, of shape (n_x, m, T_x)
                   da0 -- Gradient w.r.t. the previous hidden state, np array of shape (n_a, m)
                   dWf -- Gradient w.r.t. the weight matrix forget gate, np array of shape (n_a, n_a + n_x)
                   dWi -- Gradient w.r.t. the weight matrix update gate, np array of shape (n_a, n_a + n_x)
                   dWc -- Gradient w.r.t. the weight matrix memory gate, np array of shape (n_a, n_a + n_x)
                   dWo -- Gradient w.r.t. the weight matrix save gate, np array of shape (n_a, n_a + n_x)
                   dbf -- Gradient w.r.t. biases of the forget gate, of shape (n_a, 1)
                   dbi -- Gradient w.r.t. biases of the update gate, of shape (n_a, 1)
                   dbc -- Gradient w.r.t. biases of the memory gate, of shape (n_a, 1)
                   dbo -- Gradient w.r.t. biases of the save gate, of shape (n_a, 1)
    """

    # Retrieve values from the first cache (t=1) of caches.
    (caches, x) = caches
    (a1, c1, a0, c0, f1, i1, cc1, o1, x1, params) = caches[0] # at t=1
    
    ### CODE HERE:
    # Retrieve dimensions from da's and x1's shapes
    (n_a, m, T_x), (n_x, m) = da.shape, x1.shape
    
    # initialize the gradients with the right sizes
    dx  = np.zeros(x.shape)       # == np.zeros((n_x, m, T_x))
    da0 = np.zeros((n_a, m))
    da_prev_t = np.zeros((n_a, m)) # np.zeros(da.shape) # np.zeros((n_a, m)) 
    dc_prev_t = np.zeros((n_a, m)) # np.zeros(da.shape) # == np.zeros((n_a, m, T_x))
    
    dWf = np.zeros((n_a, n_a + n_x))
    dWi = np.zeros((n_a, n_a + n_x))
    dWc = np.zeros((n_a, n_a + n_x))
    dWo = np.zeros((n_a, n_a + n_x))
    
    dbf = np.zeros((n_a, 1))
    dbi = np.zeros((n_a, 1))
    dbc = np.zeros((n_a, 1))
    dbo = np.zeros((n_a, 1))
    
    # loop back over the whole sequence
    for t in reversed(range(T_x)):
        gradients = lstm_cell_backward(da[:, :, t] + da_prev_t, 
                                       dc_prev_t, 
                                       caches[t]) # Compute all gradients using lstm_cell_backward
        
        # gradients = {"dxt": dxt, "da_prev": da_prev, "dc_prev": dc_prev, "dWf": dWf,"dbf": dbf, "dWi": dWi, 
        #              "dbi": dbi, "dWc": dWc, "dbc": dbc, "dWo": dWo, "dbo": dbo}
                            
        # Store or add the gradient to the parameters' previous step's gradient
        da_prev_t = gradients['da_prev']
        dc_prev_t = gradients['dc_prev']            
        dx[:, :, t] = gradients['dxt'] # stored

        dWf += gradients['dWf']
        dWi += gradients['dWi']
        dWc += gradients['dWc']
        dWo += gradients['dWo']

        dbf += gradients['dbf']
        dbi += gradients['dbi']
        dbc += gradients['dbc']
        dbo += gradients['dbo']

    # Set the first activation's gradient to the backpropagated gradient da_prev.
    da0 = gradients['da_prev']
    
    # Store the gradients in a python dictionary
    gradients = {"dx": dx, "da0": da0, "dWf": dWf, "dbf": dbf, "dWi": dWi, "dbi": dbi,
                "dWc": dWc, "dbc": dbc, "dWo": dWo, "dbo": dbo}
    
    return gradients

In [94]:
np.random.seed(1)
x_tmp = np.random.randn(3, 10, 7) # NOTE: should be: (3, 10, 7) - but then does not match expected output
a0_tmp = np.random.randn(5, 10)

params_tmp = {}
params_tmp['Wf'] = np.random.randn(5, 5 + 3)
params_tmp['bf'] = np.random.randn(5,1)
params_tmp['Wi'] = np.random.randn(5, 5 + 3)
params_tmp['bi'] = np.random.randn(5,1)
params_tmp['Wo'] = np.random.randn(5, 5 + 3)
params_tmp['bo'] = np.random.randn(5,1)
params_tmp['Wc'] = np.random.randn(5, 5 + 3)
params_tmp['bc'] = np.random.randn(5,1)
params_tmp['Wy'] = np.zeros((2, 5))       # unused, but needed for lstm_forward
params_tmp['by'] = np.zeros((2, 1))       # unused, but needed for lstm_forward

a_tmp, y_tmp, c_tmp, caches_tmp = lstm_forward(x_tmp, a0_tmp, params_tmp)

da_tmp = np.random.randn(5, 10, 4)
gradients_tmp = lstm_backward(da_tmp, caches_tmp)

print("gradients[\"dx\"][1][2] =", gradients_tmp["dx"][1][2])
print("gradients[\"dx\"].shape =", gradients_tmp["dx"].shape)
print("gradients[\"da0\"][2][3] =", gradients_tmp["da0"][2][3])
print("gradients[\"da0\"].shape =", gradients_tmp["da0"].shape)
print()
print("gradients[\"dWf\"][3][1] =", gradients_tmp["dWf"][3][1])
print("gradients[\"dWf\"].shape =", gradients_tmp["dWf"].shape)
print("gradients[\"dWi\"][1][2] =", gradients_tmp["dWi"][1][2])
print("gradients[\"dWi\"].shape =", gradients_tmp["dWi"].shape)
print("gradients[\"dWc\"][3][1] =", gradients_tmp["dWc"][3][1])
print("gradients[\"dWc\"].shape =", gradients_tmp["dWc"].shape)
print("gradients[\"dWo\"][1][2] =", gradients_tmp["dWo"][1][2])
print("gradients[\"dWo\"].shape =", gradients_tmp["dWo"].shape)
print()
print("gradients[\"dbf\"][4] =", gradients_tmp["dbf"][4])
print("gradients[\"dbf\"].shape =", gradients_tmp["dbf"].shape)
print("gradients[\"dbi\"][4] =", gradients_tmp["dbi"][4])
print("gradients[\"dbi\"].shape =", gradients_tmp["dbi"].shape)
print("gradients[\"dbc\"][4] =", gradients_tmp["dbc"][4])
print("gradients[\"dbc\"].shape =", gradients_tmp["dbc"].shape)
print("gradients[\"dbo\"][4] =", gradients_tmp["dbo"][4])
print("gradients[\"dbo\"].shape =", gradients_tmp["dbo"].shape)

gradients["dx"][1][2] = [ 0.00218254  0.28205375 -0.48292508 -0.43281115  0.          0.
  0.        ]
gradients["dx"].shape = (3, 10, 7)
gradients["da0"][2][3] = 0.3127703102572603
gradients["da0"].shape = (5, 10)

gradients["dWf"][3][1] = -0.08098023109383466
gradients["dWf"].shape = (5, 8)
gradients["dWi"][1][2] = 0.4051243309298186
gradients["dWi"].shape = (5, 8)
gradients["dWc"][3][1] = -0.07937467355121491
gradients["dWc"].shape = (5, 8)
gradients["dWo"][1][2] = 0.03894877576298697
gradients["dWo"].shape = (5, 8)

gradients["dbf"][4] = [-0.15745657]
gradients["dbf"].shape = (5, 1)
gradients["dbi"][4] = [-0.50848333]
gradients["dbi"].shape = (5, 1)
gradients["dbc"][4] = [-0.42510818]
gradients["dbc"].shape = (5, 1)
gradients["dbo"][4] = [-0.17958196]
gradients["dbo"].shape = (5, 1)


**Expected Output**:

<table>
    <tr>
        <td>
            <b>gradients["dx"][1][2]</b>=
        </td>
        <td>
            [0.00218254  0.28205375 -0.48292508 -0.43281115 <it>0. 0. 0.</it>]
        </td>
    </tr>
        <tr>
        <td>
            <b>gradients["dx"].shape</b>=
        </td>
        <td>
            (3, 10, <it>7</it>)
        </td>
    </tr>
        <tr>
        <td>
            <b>gradients["da0"][2][3]</b>=
        </td>
        <td>
           0.312770310257
        </td>
    </tr>
        <tr>
        <td>
            <b>gradients["da0"].shape</b>=
        </td>
        <td>
           (5, 10)
        </td>
    </tr>
    <tr><td>&nbsp;</td></tr>
        <tr>
        <td>
            <b>gradients["dWf"][3][1]</b>= 
        </td>
        <td>
           -0.0809802310938
        </td>
    </tr>
        <tr>
        <td>
            <b>gradients["dWf"].shape</b>=
        </td>
        <td>
           (5, 8)
        </td>
    </tr>
        <tr>
        <td>
            <b>gradients["dWi"][1][2]</b>= 
        </td>
        <td>
           0.40512433093
        </td>
    </tr>
        <tr>
        <td>
            <b>gradients["dWi"].shape</b>= 
        </td>
        <td>
           (5, 8)
        </td>
    </tr>
    <tr>
        <td>
            <b>gradients["dWc"][3][1]</b>= 
        </td>
        <td>
           -0.0793746735512
        </td>
    </tr>
        <tr>
        <td>
            <b>gradients["dWc"].shape</b>= 
        </td>
        <td>
           (5, 8)
        </td>
    </tr>
    <tr>
        <td>
            <b>gradients["dWo"][1][2]</b>= 
        </td>
        <td>
           0.038948775763
        </td>
    </tr>
    <tr><td>&nbsp;</td></tr>
        <tr>
        <td>
            <b>gradients["dWo"].shape</b>= 
        </td>
        <td>
           (5, 8)
        </td>
    </tr>
    <tr>
        <td>
            <b>gradients["dbf"][4]</b>= 
        </td>
        <td>
           [-0.15745657]
        </td>
    </tr>
        <tr>
        <td>
            <b>gradients["dbf"].shape</b>= 
        </td>
        <td>
           (5, 1)
        </td>
    </tr>
    <tr>
        <td>
            <b>gradients["dbi"][4]</b>= 
        </td>
        <td>
           [-0.50848333]
        </td>
    </tr>
        <tr>
        <td>
            <b>gradients["dbi"].shape</b>= 
        </td>
        <td>
           (5, 1)
        </td>
    </tr>
        <tr>
        <td>
            <b>gradients["dbc"][4]</b>= 
        </td>
        <td>
           [-0.42510818]
        </td>
    </tr>
        <tr>
        <td>
            <b>gradients["dbc"].shape</b>= 
        </td>
        <td>
           (5, 1)
        </td>
    </tr>
        <tr>
        <td>
            <b>gradients["dbo"][4]</b>= 
        </td>
        <td>
           [ -0.17958196]
        </td>
    </tr>
        <tr>
        <td>
            <b>gradients["dbo"].shape</b>= 
        </td>
        <td>
           (5, 1)
        </td>
    </tr>
</table>

### Congratulations !

Congratulations on completing this assignment. You now understand how recurrent neural networks work! 

Let's go on to the next exercise, where you'll use an RNN to build a character-level language model.